In [1]:
import os
import csv
import keras
import numpy as np
import pandas as pd
from scipy import misc
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import pickle

%matplotlib inline

Using TensorFlow backend.
/home/li/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
images_left = []
images_center = []
images_right = []
measurement = []

log = pd.read_csv('data-all/driving_log.csv')
log = log.applymap(lambda x: x.strip() if isinstance(x, str) else x)
log_shape = log.shape
log_columns = log.columns.values
print(log_shape)
print(log_columns)
print(log.dtypes)

(21509, 7)
['center' 'left' 'right' 'steering' 'throttle' 'brake' 'speed']
center       object
left         object
right        object
steering    float64
throttle    float64
brake       float64
speed       float64
dtype: object


In [3]:
def extract(img_path):
    img = cv2.imread(img_path)
    img = img[...,::-1]
    return img

In [4]:
lbar = tqdm(range(log.shape[0]), unit='images')
#for row_idx in range(log_shape[0]):
for row_idx in lbar:
    measurement.append([log.loc[row_idx]['steering'], 
                   log.loc[row_idx]['throttle'], 
                   log.loc[row_idx]['brake'], 
                   log.loc[row_idx]['speed']])
    img_left = extract(os.path.join('data-all', log.loc[row_idx]['left']))
    images_left.append(img_left)
    
    img_right = extract(os.path.join('data-all', log.loc[row_idx]['right']))
    images_right.append(img_right)
    
    img_center = extract(os.path.join('data-all', log.loc[row_idx]['center']))
    images_center.append(img_center)
images_left = np.array(images_left)
images_right = np.array(images_right)
images_center = np.array(images_center)

100%|██████████| 21509/21509 [00:30<00:00, 705.11images/s]


In [5]:
driving_data = {'images_left': images_left, 'images_right': images_right, 'images_center': images_center, 'log': log}
#driving_data = {'images_left': images_left, 'log': log}
with open('driving_data.pickle', 'wb') as pfile:
    pickle.dump(driving_data, pfile, pickle.HIGHEST_PROTOCOL)
    del driving_data

In [7]:
with open('driving_data.pickle', 'rb') as pfile:
    driving_data = pickle.load(pfile)